In [1]:
from pybaseball import statcast_pitcher
from pybaseball import playerid_lookup
import pandas as pd
import numpy as np

In [2]:
#playerid_lookup('verlander')

In [3]:
verlander_18_df = statcast_pitcher('2018-04-01', '2018-09-30', 434378)

Gathering Player Data


In [4]:
verlander_18_df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,CU,2018-09-29,81.4,-1.85,6.69,"Verlander, Justin",430945,434378,field_out,hit_into_play,...,3,3,0,0,3,Standard,Standard,25.0,-0.024,-0.161
1,CU,2018-09-29,81.4,-1.79,6.66,"Verlander, Justin",430945,434378,NaN,blocked_ball,...,3,3,0,0,3,Standard,Standard,27.0,0.000,0.028
2,CU,2018-09-29,81.2,-1.74,6.67,"Verlander, Justin",430945,434378,NaN,called_strike,...,3,3,0,0,3,Standard,Standard,23.0,0.000,-0.050
3,FF,2018-09-29,96.9,-1.62,6.88,"Verlander, Justin",430945,434378,NaN,foul,...,3,3,0,0,3,Standard,Standard,211.0,0.000,-0.039
4,FF,2018-09-29,95.9,-1.72,6.98,"Verlander, Justin",542340,434378,walk,ball,...,3,3,0,0,3,Standard,Standard,206.0,0.014,0.102


In [5]:
verlander_18_df.shape

(3337, 92)

In [6]:
verlander_18_df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [7]:
verlander_short = verlander_18_df.head(50)

In [8]:
#verlander_short.to_csv('test.csv')

In [9]:
def create_sequence_id_at_bat_ordered(df, num_pitches=3):
    """
    Create a 'sequence id' column in the DataFrame that contains information about the previous
    `num_pitches` pitches in the at-bat for the same batter, in chronological order. The information 
    includes Count, Pitch type, Location, and Result. If the at-bat has had fewer than `num_pitches`, 
    return 0.

    Args:
    df (DataFrame): The baseball data DataFrame.
    num_pitches (int): Number of previous pitches to include in the sequence.

    Returns:
    DataFrame: Modified DataFrame with the 'sequence id' column.
    """
    # Initialize a new column for the sequence id
    df['sequence_id'] = None

    # Iterate through the DataFrame
    for i in range(len(df)):
        # Initialize an empty list to store sequence information
        sequence_info = []
        current_batter = df.iloc[i]['batter']

        # Collect information from the previous `num_pitches` pitches for the same batter
        for j in range(1, num_pitches + 1):
            # Ensure we don't go out of bounds and the batter is the same
            if i - j >= 0 and df.iloc[i - j]['batter'] == current_batter:
                # Extract relevant information for the pitch
                pitch_info = df.iloc[i - j]
                count = str(pitch_info['balls']) + '-' + str(pitch_info['strikes'])
                pitch_type = pitch_info['pitch_type']
                location = pitch_info['zone']
                result = pitch_info['description']

                # Create a string representation of the pitch information
                pitch_sequence = f"{count}_{pitch_type}_{location}_{result}"

                # Append to the sequence information list
                sequence_info.append(pitch_sequence)
            else:
                # If the at-bat has not had `num_pitches`, return 0
                sequence_info.append('0')
                break

        # Maintain chronological order of the sequence
        sequence_id_str = '|'.join(sequence_info)  # Keep the original order
        df.at[i, 'sequence_id'] = sequence_id_str

    return df

# Apply the function to create a sequence id based on at-bat history for the last three pitches
df_with_at_bat_sequence_ordered = create_sequence_id_at_bat_ordered(verlander_18_df.iloc[::-1], num_pitches=3)

# Display the first few rows of the modified DataFrame
df_with_at_bat_sequence_ordered[['batter', 'sequence_id', 'balls', 'strikes', 'description']].head()


/var/folders/px/v7992sbn36q0sd74xlm965340000gn/T/ipykernel_48604/913996268.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sequence_id'] = None


,batter,sequence_id,balls,strikes,description
3336,448801,0-2_CU_14.0_blocked_ball|0-1_CU_4.0_called_str...,0,0,ball
3335,448801,0-1_CU_4.0_called_strike|0-0_FF_5.0_foul|0,1,0,called_strike
3334,448801,0-0_FF_5.0_foul|0,1,1,called_strike
3333,448801,0,1,2,foul
3332,448801,3-1_FF_5.0_swinging_strike|3-0_FF_5.0_called_s...,1,2,swinging_strike


In [10]:
verlander_18_df.shape

(3337, 92)

In [11]:
# get a list of pitchers that match criteria
## min pitches

In [12]:
from pybaseball import statcast_pitcher_percentile_ranks
from pybaseball import pitching_stats

In [13]:
data = pitching_stats(2023, qual=0)
data.head(50)

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
252,10310,2023,Zack Wheeler,PHI,33,13,6,5.9,3.61,32,...,NaN,NaN,NaN,NaN,109.0,106.0,111.0,NaN,NaN,NaN
304,27498,2023,Spencer Strider,ATL,24,20,5,5.5,3.86,32,...,NaN,NaN,NaN,NaN,125.0,104.0,112.0,NaN,NaN,NaN
181,14107,2023,Kevin Gausman,TOR,32,12,9,5.3,3.16,31,...,NaN,105.0,108.0,109.0,105.0,104.0,105.0,NaN,NaN,NaN
127,12768,2023,Sonny Gray,MIN,33,8,8,5.3,2.79,32,...,102.0,NaN,NaN,NaN,103.0,101.0,104.0,NaN,NaN,NaN
233,19291,2023,Zac Gallen,ARI,27,17,9,5.2,3.47,34,...,95.0,NaN,NaN,NaN,106.0,104.0,107.0,NaN,NaN,NaN
111,13125,2023,Gerrit Cole,NYY,32,15,4,5.2,2.63,33,...,102.0,NaN,NaN,NaN,121.0,104.0,109.0,NaN,NaN,NaN
167,17312,2023,Justin Steele,CHC,27,16,5,4.9,3.06,30,...,NaN,NaN,NaN,NaN,99.0,102.0,102.0,NaN,NaN,NaN
194,17995,2023,Logan Webb,SFG,26,11,13,4.9,3.25,33,...,NaN,NaN,NaN,NaN,104.0,104.0,103.0,NaN,NaN,NaN
235,13774,2023,Zach Eflin,TBR,29,16,8,4.8,3.50,31,...,102.0,NaN,NaN,NaN,98.0,108.0,106.0,NaN,NaN,NaN
267,17085,2023,Pablo Lopez,MIN,27,11,8,4.5,3.66,32,...,NaN,NaN,NaN,NaN,99.0,107.0,109.0,NaN,NaN,NaN


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 863 entries, 252 to 638
Columns: 393 entries, IDfg to Pit+ FO
dtypes: float64(328), int64(61), object(4)
memory usage: 2.6+ MB


In [15]:
data.dropna(axis=1, inplace=True)

In [16]:
data.head()

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xERA
252,10310,2023,Zack Wheeler,PHI,33,13,6,5.9,3.61,32,...,14.1,27,0.051,113.2,191,0.362,527,0.143,0.276,3.21
304,27498,2023,Spencer Strider,ATL,24,20,5,5.5,3.86,32,...,17.0,33,0.080,111.8,147,0.354,415,0.149,0.338,3.09
181,14107,2023,Kevin Gausman,TOR,32,12,9,5.3,3.16,31,...,12.3,46,0.098,117.9,202,0.432,468,0.167,0.296,3.87
127,12768,2023,Sonny Gray,MIN,33,8,8,5.3,2.79,32,...,9.6,35,0.069,113.7,199,0.390,510,0.166,0.279,3.69
233,19291,2023,Zac Gallen,ARI,27,17,9,5.2,3.47,34,...,12.3,53,0.092,113.8,264,0.460,574,0.173,0.285,4.18


In [17]:
data.columns

Index(['IDfg', 'Season', 'Name', 'Team', 'Age', 'W', 'L', 'WAR', 'ERA', 'G',
       ...
       'LA', 'Barrels', 'Barrel%', 'maxEV', 'HardHit', 'HardHit%', 'Events',
       'CStr%', 'CSW%', 'xERA'],
      dtype='object', length=142)

In [18]:
#data.to_csv('../test.csv')

In [19]:
cols = ['Name', 'Pitches', 'IP']
pitcher_list = data[cols]

In [20]:
pitcher_list.describe()

,Pitches,IP
count,863.000000,863.000000
mean,833.170336,49.705678
std,805.992081,49.420555
min,4.000000,0.100000
25%,194.500000,10.600000
50%,613.000000,35.200000
75%,1121.000000,67.100000
max,3286.000000,216.000000


In [21]:
863 * .25

215.75

In [22]:
df_sorted = pitcher_list.sort_values(by='IP', ascending=False)

In [23]:
top_25_percent_index = int(len(df_sorted) * 0.15)

# Extract the names of the top 25% pitchers
top_pitchers = df_sorted.iloc[:top_25_percent_index]['Name']

# Iterate through the names and print them
selected_pitchers = []
for name in top_pitchers:
    selected_pitchers.append(name)

In [24]:
len(selected_pitchers)

129

In [25]:
#selected_pitchers = [name.split()[-1] for name in selected_pitchers]

In [26]:
# for loop that iterates through selected pitchers in statcast pitcher to create new dataframe for generalization

In [27]:
last_selected_pitchers = [name.split()[-1] for name in selected_pitchers]
first_selected_pitchers = [name.split()[0] for name in selected_pitchers]
zip(first_selected_pitchers, last_selected_pitchers)

In [28]:
list = zip(first_selected_pitchers, last_selected_pitchers)
result = set(list)
result

{('Aaron', 'Civale'),
 ('Aaron', 'Nola'),
 ('Adam', 'Wainwright'),
 ('Adrian', 'Houser'),
 ('Alex', 'Cobb'),
 ('Andrew', 'Abbott'),
 ('Andrew', 'Heaney'),
 ('Anthony', 'DeSclafani'),
 ('Austin', 'Gomber'),
 ('Bailey', 'Ober'),
 ('Blake', 'Snell'),
 ('Bobby', 'Miller'),
 ('Brady', 'Singer'),
 ('Brandon', 'Williamson'),
 ('Braxton', 'Garrett'),
 ('Brayan', 'Bello'),
 ('Bryce', 'Elder'),
 ('Bryce', 'Miller'),
 ('Cal', 'Quantrill'),
 ('Charlie', 'Morton'),
 ('Chris', 'Bassitt'),
 ('Chris', 'Flexen'),
 ('Chris', 'Sale'),
 ('Clarke', 'Schmidt'),
 ('Clayton', 'Kershaw'),
 ('Colin', 'Rea'),
 ('Corbin', 'Burnes'),
 ('Cristian', 'Javier'),
 ('Dane', 'Dunning'),
 ('David', 'Peterson'),
 ('Dean', 'Kremer'),
 ('Domingo', 'German'),
 ('Drew', 'Smyly'),
 ('Dylan', 'Cease'),
 ('Eduardo', 'Rodriguez'),
 ('Framber', 'Valdez'),
 ('Freddy', 'Peralta'),
 ('George', 'Kirby'),
 ('Gerrit', 'Cole'),
 ('Graham', 'Ashcraft'),
 ('Grayson', 'Rodriguez'),
 ('Griffin', 'Canning'),
 ('Hunter', 'Brown'),
 ('Hunter', '

In [29]:
player_ids = []

# Iterate through the list of player name tuples
for first_name, last_name in result:
    try:
        # Use playerid_lookup function with first and last names
        player_id_df = playerid_lookup(last_name, first_name)

        # Check if the DataFrame is not empty and add to the list
        if player_id_df is not None and not player_id_df.empty:
            player_ids.append(player_id_df)
    except Exception as e:
        print(f"Error fetching player ID for {first_name} {last_name}: {e}")

# Optional: Combine all the individual DataFrames into one
combined_player_ids = pd.concat(player_ids, ignore_index=True)

Gathering player lookup table. This may take a moment.


In [30]:
combined_player_ids = combined_player_ids['key_mlbam']
combined_player_ids

0      458681
1      605288
2      607192
3      671106
4      663531
        ...  
117    657277
118    669194
119    605182
120    425844
121    680570
Name: key_mlbam, Length: 122, dtype: int64

In [31]:
combined_df = pd.DataFrame()

# Iterate through each pitcher's name and fetch their data
for id in combined_player_ids:
    try:
        # Fetch data for the pitcher
        pitcher_df = statcast_pitcher('2023-04-01', '2023-09-30', id)

        # Check if the DataFrame is not empty
        if pitcher_df is not None and not pitcher_df.empty:
            # Append the data to the combined DataFrame
            combined_df = pd.concat([combined_df, pitcher_df])
    except Exception as e:
        print(f"Error fetching data for {name}: {e}")

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

In [32]:
combined_df.to_csv('../top_129_pitchers.csv')